# PCB Placement Training with TensorFlow on Cloud ML Engine

This notebook demonstrates how to do PCB placement training from scratch using a placement dataset using Estimator/Experiment.

In [ ]:
import os
PROJECT = 'gcp-spb-magestic' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'joydeeps-datastore' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE = 'cnn'

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_TYPE'] = MODEL_TYPE

In [ ]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## Data Preprocessing, Bin the scores
Bucketize the scores into 9 bins between 0 and 1

In [ ]:
%bash
export DATALAB_DATASET=${PWD}/pcb_dataset
export DATASET=gs://${BUCKET}/pcb_dataset

In [ ]:
import csv
import numpy as np
import pandas as pd
from io import BytesIO
import StringIO


def transformFileName(imageName):
  return("gs://" + BUCKET + "/pcb_dataset/images/" + imageName)

labelFileName = "gs://" + BUCKET + "/pcb_dataset/images/labelmap.txt"
trainFileName = "gs://" + BUCKET + "/pcb_dataset/images/train.csv"
evalFileName = "gs://" + BUCKET + "/pcb_dataset/images/eval.csv"

%gcs read --object $labelFileName --variable csv_as_bytes

labeldf = pd.read_csv(BytesIO(csv_as_bytes), names=['type','filename','score'])
scores = labeldf['score']
bins = np.linspace(0, 1, 10,endpoint=False)
digitized = np.digitize(scores, bins)
labeldf['score'] = digitized
traindf = labeldf[labeldf['type']=='train']
evaldf = labeldf[labeldf['type']=='test']
traindf = traindf[['filename','score']]
evaldf = evaldf[['filename','score']]
traindf['filename'] = traindf['filename'].apply(transformFileName)
evaldf['filename'] = evaldf['filename'].apply(transformFileName)
traincsv=StringIO.StringIO()
evalcsv=StringIO.StringIO()
traindf.to_csv(path_or_buf=traincsv,header=None,index=False)
evaldf.to_csv(path_or_buf=evalcsv,header=None,index=False)
trainstr = traincsv.getvalue()
evalstr = evalcsv.getvalue()

%gcs write --object $trainFileName --variable trainstr
%gcs write --object $evalFileName --variable evalstr


## Invoke Tensorboard on output directory

In [ ]:
from google.datalab.ml import TensorBoard
OUTDIR = 'gs://' +BUCKET + '/export_model_mle'
TensorBoard().start(OUTDIR)

## Run as a Python module

Let's run it as Python module.  Note the --model parameter

In [ ]:
%bash
export OUTDIR=gs://${BUCKET}/export_model
export DATADIR=gs://${BUCKET}/images
export PYTHONPATH=${PYTHONPATH}:${PWD}/placermodel
python -m trainer.task --output_dir=$OUTDIR --dataset_dir=$DATADIR \
   --train_steps=6000 --learning_rate=0.01 --train_batch_size=40 \
   --model=$MODEL_TYPE --batch_norm

Now, let's do it on ML Engine. Note the --model parameter

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/export_model_mle
JOBNAME=pcb_placer_${MODEL_TYPE}_$(date -u +%y%m%d_%H%M%S)
CONFIG=gs://${BUCKET}/config.yaml
export DATADIR=gs://${BUCKET}/pcb_dataset/images
echo $CONFIG $DATADIR $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
#gsutil -m cp $CONFIG ${PWD}/config.yaml
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/placermodel/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --runtime-version=1.2 \
   --config=${PWD}/config.yaml \
   -- \
   --dataset_dir=$DATADIR \
   --output_dir=$OUTDIR \
   --train_steps=5000 --learning_rate=0.01 --train_batch_size=40 \
   --model=$MODEL_TYPE --batch_norm

## Deploying and predicting with model

Deploy the model:

In [ ]:
%bash
MODEL_NAME="placermodel1"
#MODEL_VERSION=${MODEL_TYPE}
MODEL_VERSION="cnn"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/export_model_mle/export/Servo | tail -1)

echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
gcloud ml-engine versions delete --quiet ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=1.2

In [ ]:
%writefile test.json
{"imageurl": "gs://joydeeps-datastore/pcb_dataset/images/study.1.png"}

Send it to the prediction service

In [ ]:
%bash
gcloud ml-engine predict --model=placermodel --version=${MODEL_TYPE} --json-instances=./test.json

In [ ]:
%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/export_model_mle/export/Servo | tail -1)
gcloud ml-engine local predict --model-dir=$MODEL_LOCATION --json-instances=test.json